In [2]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from scipy.io import wavfile
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [3]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [4]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v6\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [5]:
print(len(val_uids))

2772


In [6]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
# from utils.auto_tqdm import tqdm
from evaluate import evaluate

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v6/0703a_preprocess.yaml -m ./config/Ego4D_final_v6/0703a_model.yaml -t ./config/Ego4D_final_v6/0703a_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [9]:
pprint(args)
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v6/0703a_preprocess.yaml', model_config='./config/Ego4D_final_v6/0703a_model.yaml', train_config='./config/Ego4D_final_v6/0703a_train.yaml')
Prepare training ...


In [10]:
ckpt_path = r'./output/LibriTTS/LibriTTS_800000.pth.tar'
ckpt = torch.load(ckpt_path)

In [16]:
pprint([n for n in ckpt['model'].keys()])

['encoder.position_enc',
 'encoder.src_word_emb.weight',
 'encoder.layer_stack.0.slf_attn.w_qs.weight',
 'encoder.layer_stack.0.slf_attn.w_qs.bias',
 'encoder.layer_stack.0.slf_attn.w_ks.weight',
 'encoder.layer_stack.0.slf_attn.w_ks.bias',
 'encoder.layer_stack.0.slf_attn.w_vs.weight',
 'encoder.layer_stack.0.slf_attn.w_vs.bias',
 'encoder.layer_stack.0.slf_attn.layer_norm.weight',
 'encoder.layer_stack.0.slf_attn.layer_norm.bias',
 'encoder.layer_stack.0.slf_attn.fc.weight',
 'encoder.layer_stack.0.slf_attn.fc.bias',
 'encoder.layer_stack.0.pos_ffn.w_1.weight',
 'encoder.layer_stack.0.pos_ffn.w_1.bias',
 'encoder.layer_stack.0.pos_ffn.w_2.weight',
 'encoder.layer_stack.0.pos_ffn.w_2.bias',
 'encoder.layer_stack.0.pos_ffn.layer_norm.weight',
 'encoder.layer_stack.0.pos_ffn.layer_norm.bias',
 'encoder.layer_stack.1.slf_attn.w_qs.weight',
 'encoder.layer_stack.1.slf_attn.w_qs.bias',
 'encoder.layer_stack.1.slf_attn.w_ks.weight',
 'encoder.layer_stack.1.slf_attn.w_ks.bias',
 'encoder.lay

In [18]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)

=> Using speaker embeddings.
True
2
=> Using VarianceAdaptorWithSpeaker.


In [20]:
print([n for n, _ in model.named_children()])

['encoder', 'variance_adaptor', 'decoder', 'mel_linear', 'postnet', 'speaker_layernorm']


In [21]:
[n for n, _ in model.named_children()]

['encoder',
 'variance_adaptor',
 'decoder',
 'mel_linear',
 'postnet',
 'speaker_layernorm']

In [ ]:
model.load_state_dict(ckpt["model"], strict=False)

In [26]:
for param in model.encoder.parameters():
    param.requires_grad_(False)
for param in model.decoder.parameters():
    param.requires_grad_(False)
for param in model.mel_linear.parameters():
    param.requires_grad_(False)
for param in model.postnet.parameters():
    param.requires_grad_(False)

In [ ]:
model.to(device)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)